In [ ]:
from utliz.jm_utilz import *
from utliz.jira_config import *

import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# Suppress only the InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Migration Process Steps:
1. Disable on-prem ability to submit new records or update current.
2. Use the on-prem migration assistant to migrate only users/groups
3. While user/groups are being migrated, use the utliz.jm_delete.py to delete all issue records in the cloud instance
4. After the jm_delete process has started deleting records, start the CSV build process
5. After all cloud records have been deleted and the new CSV has been built, load the new CSV into the cloud
6. Validate all CSV records have been migrated to the cloud
7. After the CSV has been loaded, update the comments to the on-prem state then load attachements then update the Google reference docs  
8. Update all required DNS to point to new instance 

## Create a database connetion object that is used to extract the required data from views built from the backend Jira tables

In [ ]:
_sys_info_services = SysInfoServices(jconfig['db_server'],
                                     jconfig['database'],
                                     jconfig['db_username'],
                                     jconfig['db_password'])


api_token = jconfig['api_token']
url = jconfig['url']
jira_session = Jira(url, jconfig['jira_user_name'], api_token)

# Build Custom CSV to Load into the Cloud
### Note: comments made in code to help identify locations for custom updates 

In [ ]:
# load the mirgration view and format the data 
##### ------------------> this assumes the SQL tables were built using the files found in the SQL folder so the view names listed below would match those records
all_issues = _sys_info_services.get_table_or_view_data('vw_all_jira_issues') # all jira issues
all_issues['Issue_ID'] = all_issues['Issue_ID'].astype(int)
all_issues['Issue_Ticket_Number'] = all_issues['Issue_Ticket_Number'].astype(int)
all_issues['Issue_Priority'] = all_issues['Issue_Priority'].astype(int)
all_issues['Issue_Status_Category'] = all_issues['Issue_Status_Category'].astype(int)
all_issues['Parent_Issue_ID'] =  pd.to_numeric(all_issues['Parent_Issue_ID'], errors='coerce').astype('Int64')
all_issues['TIMEORIGINALESTIMATE'] =  pd.to_numeric(all_issues['TIMEORIGINALESTIMATE'], errors='coerce').astype('Int64')
all_issues['TIMEESTIMATE'] =  pd.to_numeric(all_issues['TIMEESTIMATE'], errors='coerce').astype('Int64')
all_issues['TIMESPENT'] =  pd.to_numeric(all_issues['TIMESPENT'], errors='coerce').astype('Int64')

fields_to_drop = ['Assignee_User_Is_Active',
                  'Creator_User_Is_Active',
                  'Issue_Status_ID',
                  'Issue_Status_Description',
                  'Issue_Status_Category',
                  'Issue_Status_Icon_Relative_Url',
                  'Issue_Type_Is_SubTask',
                  'Issue_Type_Icon_Relative_Url',
                  'Resolution_ID',
                  'Reporter_User_Is_Active',
                  'Issue_Priority',
                  'Assignee_User_Display_Name',
                  'Creator_UserName',
                  'Creator_User_Display_Name',
                  #'Issue_Description',
                  'Issue_Relative_URL',
                  'Issue_Type_Description',
                  'Issue_Type_ID',
                  'Resolution_Description']

all_issues = all_issues.drop(columns=fields_to_drop)




cf_data = _sys_info_services.get_table_or_view_data('vw_custom_field_data') # custom field data for issues
cf_data['custom_field_value_record_id'] = cf_data['custom_field_value_record_id'].astype(int)
cf_data['ISSUE'] = cf_data['ISSUE'].astype(int)
cf_data['CUSTOMFIELD'] = cf_data['CUSTOMFIELD'].astype(int)
cf_data['ID'] = cf_data['ID'].astype(int)
cf_data['PARENTKEY'] =  pd.to_numeric(cf_data['PARENTKEY'], errors='coerce').astype('Int64')

cf_options = _sys_info_services.get_table_or_view_data('vw_custom_field_options') # mapping table to determine what the human readable translation of coded values
cf_options['ID'] = cf_options['ID'].astype(int)
cf_options['CUSTOMFIELD'] = cf_options['CUSTOMFIELD'].astype(int)
cf_options['CUSTOMFIELDCONFIG'] = cf_options['CUSTOMFIELDCONFIG'].astype(int)
cf_options['SEQUENCE'] = cf_options['SEQUENCE'].astype(int)


watchers = _sys_info_services.get_table_or_view_data('vw_jira_watchers')
watchers['Issue_ID'] = watchers['issue_id'].astype(int)


worklogsdf = _sys_info_services.get_table_or_view_data('vw_all_worklogs')
worklogsdf['issueid'] = worklogsdf['issueid'].astype(int)
worklogsdf['ID'] = worklogsdf['ID'].astype(int) 
worklogsdf = update_time_for_worklog(worklogsdf)
worklogsdf['STARTDATE'] = worklogsdf['STARTDATE'].astype(str) 


commentsdf = _sys_info_services.get_table_or_view_data('vw_jira_comments')
commentsdf['Issue_ID'] = commentsdf['Issue_ID'].astype(int)
commentsdf['Issue_Ticket_Number'] = commentsdf['Issue_Ticket_Number'].astype(int) 
commentsdf['issuenum'] = commentsdf['issuenum'].astype(int)
commentsdf['unique_comment_id'] = commentsdf['unique_comment_id'].astype(int)
commentsdf['Parent_Issue_ID'] =  pd.to_numeric(commentsdf['Parent_Issue_ID'], errors='coerce').astype('Int64')


# loop through the folder that contains the migration records sets from users and extract the contents
issue_folder_path = jconfig['issue_folder_path'] # path where excel files that contain records provided by agents for what jira records they need migrated 
issue_error_path = jconfig['issue_error_path'] # path where error reports will be stored 
issue_file_list = list_files_in_folder(issue_folder_path)
issue_errors_list = list_files_in_folder(issue_error_path)




# delete error records if they exist
if len(issue_errors_list) > 0:
    print('deleting prior error files')
    for i in issue_errors_list:
        delete_file(i['file_path'])
else:
    print('no prior error files found to delete')



issue_hold_df = []
for i in issue_file_list:
    # print(i['file_path'], ' <- excel path')
    df = pd.read_excel(i['file_path'])
    df = df.drop_duplicates().reset_index(drop=True)

    # qc to ensure parent records exist for a child record
    df, invalid_rows = check_if_parent_record_exists(df)  

    # if there are child records that do not have the parent record in the set the 
    # save off the error report 
    if len(invalid_rows) > 0:
        print(i['file_name'], ' missing parent records for the following file. ')
        error_path = issue_error_path+'missing_parents_'+i['file_name']
        invalid_rows.to_excel(error_path, index=False)

    issue_hold_df.append(df)


issues_concat = pd.concat(issue_hold_df)
issues_concat = issues_concat.drop_duplicates().reset_index(drop=True)
print(len(issues_concat), ' <-- Number of issues from team members')



def get_project(input_string):
    # Find the position of the hyphen
    hyphen_pos = input_string.find('-')

    # If the hyphen is found, return the substring before it
    if hyphen_pos != -1:
        return input_string[:hyphen_pos]
    else:
        # Return the original string if no hyphen is found
        return input_string


# build a project key "RDS or UTT or etc"
issues_concat['project'] = issues_concat.apply(lambda x: get_project(x['Issue Key']), axis=1)

# only projects to keep for migration 
keep_projects = ['RDS','UTT'] # <-------------------------------------------------------- This will need to be updated
issues_concat = issues_concat[issues_concat.project.isin(keep_projects)]

# get a list of the issues that require migration
target_issues_list = list(set(list(issues_concat['Issue id'])))

# get a set of the target issue records from the user supplied excel files
target_record_set = all_issues[all_issues.Issue_ID.isin(target_issues_list)].reset_index(drop=True)

# use the project keys to add bulk records to the result set, for projects where ALL records are being migrated 
project_key_list = ['BRS', 'GTT'] #  <--------------------------------------------------------------------------- This will need to be updated
filtered_all_issues = all_issues[all_issues.Project_Key.isin(project_key_list)]
target_record_set = pd.concat([target_record_set, filtered_all_issues]).reset_index(drop=True)

# now get all records created after 1/1/2024 for specific projects that were not include in the bulk load
after_date_projects = ['RDS','UTT']   #  <--------------------------------------------------------------------------- This will need to be updated
project_filter_df = all_issues[all_issues.Project_Key.isin(after_date_projects)]
project_filter_df['date_filter'] = pd.to_datetime(project_filter_df.Created_Date)
print(len(project_filter_df), ' <- number of db records before filtering for 1/1/2024 date')
target_date = pd.to_datetime('1/1/2024')
after_date_records = project_filter_df[project_filter_df.date_filter >= target_date]
after_date_records = after_date_records.drop(columns=['date_filter'])
print(len(after_date_records), ' <- number of db records after filtering for 1/1/2024 date')

print(len(target_record_set), ' <- number of excel and BRS records PRIOR to joining in records filtered for after 1/1/2024 date')
target_record_set = pd.concat([target_record_set, after_date_records]).reset_index(drop=True)
print(len(target_record_set), ' <- number of excel and BRS records AFTER to joining in records filtered for after 1/1/2024 date')

# update the following variable for downstream use
target_issues_list = list(target_record_set.Issue_ID) 

print('')
print('**************************')
print('')
print(len(target_record_set), ' <-- number of matching records from DB')
print(f'{target_record_set.Project_Key.unique()}', ' <-- project types included in the set')
print(len(target_issues_list), ' <-- sanity check to ensure the target_issues_list matches the count of matching records from the db')
print('')
print('**************************')
print('')


print('Start adding custom fields')
# get a list of the custom fields from the dict that helps map the custom names where their supporting data is found
custom_field_key_list = list(custom_field_dict)

# add the custom fields to the issues df
for f in custom_field_key_list:
    target_record_set[f] = target_record_set.apply(lambda x: build_custom_fields(x.Issue_ID, f, x.Issue_key, cf_data, cf_options, fields_needing_options_data, custom_field_dict), axis=1)

print('End adding custom fields')





print('Start update of Customer Request Type')
# apperently the downloaded csv contains a field called "Customer Request Type" that need to be updated to match the Request Type IDs found only 
# via pulling the detail from the cloud AFTER building the fields. The following replaces the values in the "Customer Request Type" field with 
# the id values from the cloud
request_types = jira_session.get_request_types(['RDS','BRS','UTT']) #  <---------------------------------------- This will need to be updated with UTT when ready
request_dict = df_create_dict(request_types, 'name', 'id')

def replace_request_type(text, request_dict):
    try:
        return int(request_dict[text])
    except:
        pass

target_record_set['Customer Request Type'] = target_record_set.apply(lambda x: replace_request_type(x['Customer Request Type'], request_dict), axis=1)
target_record_set['Customer Request Type'] =  pd.to_numeric(target_record_set['Customer Request Type'], errors='coerce').astype('Int64')
print('End updating Customer Request Type')


def contains_non_ascii(value):
    """ Check if the string contains non-ASCII characters """
    if isinstance(value, str):
        return any(ord(char) >= 128 for char in value)
    return False

def find_non_ascii_fields(df):
    non_ascii_fields = []
    for column in df.columns:
        if df[column].apply(contains_non_ascii).any():
            non_ascii_fields.append(column)
    return non_ascii_fields


print('Start formatting records')
# preprocess the record set prior to adding the duplicated fields
# update the datetime fields 
update_time_fields = ['Created_Date', 'Updated_Date', 'Resolution_Date', 'Due_Date']
for d in update_time_fields:
    target_record_set[d] = target_record_set.apply(lambda x: pd.to_datetime(x[d], errors='coerce'), axis=1)
    
for d in update_time_fields:
    target_record_set[d] = target_record_set.apply(lambda x: format_date(x[d]), axis=1)

for d in update_time_fields:
    try:
        target_record_set[d] = pd.to_datetime(target_record_set[d], errors='coerce').dt.strftime("%d/%b/%y %I:%M %p")
    except Exception as e:
        print(f"Error formatting datetime for column {d}: {e}")
        target_record_set[d] = ''  # Set an empty string for columns with formatting errors





# List of columns you want to apply the remove_non_asci function to
# columns_to_process = ['Issue_Description', 'Project Description','Issue_Status_Description','Issue_Type_Description',
# 'Issue_Summary','Project Title']# 'Change reason','Custom field (Change risk)','Custom field (Impact)', <-- No records were being returned 
columns_to_process = find_non_ascii_fields(target_record_set)


# Apply the remove_non_ascii function to the specified columns
target_record_set[columns_to_process] = target_record_set[columns_to_process].applymap(remove_non_ascii)



target_record_set = target_record_set.sort_values(by=['Parent_Issue_ID'], ascending=False, na_position='last')
target_record_set['Parent_Issue_ID'] = target_record_set['Parent_Issue_ID'].apply(lambda x: '' if pd.isna(x) else x)
target_record_set['Due_Date'] = target_record_set.apply(lambda x: '' if isinstance(x['Due_Date'], float) else x['Due_Date'], axis=1)
target_record_set['Resolution_Date'] = target_record_set.apply(lambda x: '' if isinstance(x['Resolution_Date'], float) else x['Resolution_Date'], axis=1)
target_record_set['Resolution_Name'] = target_record_set.apply(lambda x: '' if x['Resolution_Name'] == 'None' else x['Resolution_Name'], axis=1)
target_record_set['Customer Request Type'] = target_record_set['Customer Request Type'].apply(lambda x: '' if pd.isna(x) else x)
target_record_set['TIMEORIGINALESTIMATE'] = target_record_set['TIMEORIGINALESTIMATE'].apply(lambda x: '' if pd.isna(x) else x)
target_record_set['TIMEESTIMATE'] = target_record_set['TIMEESTIMATE'].apply(lambda x: '' if pd.isna(x) else x)
target_record_set['TIMESPENT'] = target_record_set['TIMESPENT'].apply(lambda x: '' if pd.isna(x) else x)



print('End formatting records')

print(len(target_record_set), ' <-- len prior to updating the watchers process')



# need to prep the watcher, worklog and comments for building out the CSV
def remove_formatting(text):
    # Define a regular expression pattern to match newline and carriage return characters
    formatting_pattern = r'[\n\r]'

    # Use the re.sub() function to replace matching patterns with an empty string
    cleaned_text = re.sub(formatting_pattern, ' ', text)

    return cleaned_text


# preprocess the comment text to import into jira so there are no ascii or other formatting issues
def clean_worklog_text(text):
    cleaned = remove_formatting(remove_non_ascii(text)).strip()
    return cleaned


# preprocess the comment text to import into jira so there are no ascii or other formatting issues
def clean_comment_text(text):
    cleaned = update_email_format(remove_formatting(remove_non_ascii(text))).strip()
    return cleaned




# filter down to the target set to make processing more efficent
watchers_target_set = watchers[watchers.issue_id.isin(target_issues_list)].reset_index(drop=True)
comments_target_set = commentsdf[commentsdf.Issue_ID.isin(target_issues_list)].reset_index(drop=True)
worklog_target_set = worklogsdf[worklogsdf.issueid.isin(target_issues_list)].reset_index(drop=True)



# preprocess the required worklog fields --> meeting and prep;16/Dec/21 9:36 PM;alolex@vcu.edu;4800
worklog_target_set['worklogbody'] = worklog_target_set.apply(lambda x: clean_worklog_text(x.worklogbody), axis=1)
#worklogsdf['comment_updated'] = worklogsdf.apply(lambda x: format_date(x.comment_updated), axis=1)

# function to preprocess the time format for values less than 60 seconds because Jira requires it in seconds but the min value 
# that can be set in a project is minutes 
worklog_target_set['timeworked'] = worklog_target_set['timeworked'].astype(int)
def minutes_to_seconds(value):
    if value < 60:
        return int(60)
    else:
        return int(value)
worklog_target_set['timeworked'] = worklog_target_set.apply(lambda x: minutes_to_seconds(x.timeworked), axis=1)
worklog_target_set['timeworked'] = worklog_target_set['timeworked'].astype(str)    




# preprocess the required commments fields
comments_target_set['comment_text'] = comments_target_set.apply(lambda x: clean_comment_text(x.comment_text), axis=1)
comments_target_set['comment_updated'] = comments_target_set.apply(lambda x: format_date(x.comment_updated), axis=1)




print(len(watchers_target_set), '  <-- len of the watcher target set')
print(len(comments_target_set), '  <-- len of the comments target set')
print(len(worklog_target_set), ' <-- len of the worklog target set')


# set a counter to determine the max amount of comments per issueid so this can be used to determine the max number of columns for the csv file
max_watchers = 0
max_comments = 0
max_worklog = 0

for l in target_issues_list:
    # filter the df based on the issueid
    target_watcher_records = watchers_target_set[watchers_target_set.issue_id == l]
    target_comment_records = comments_target_set[comments_target_set.Issue_ID == l]
    target_worklog_records = worklog_target_set[worklog_target_set.issueid == l]

    if len(target_watcher_records) > max_watchers:
        max_watchers = len(target_watcher_records)

    if len(target_comment_records) > max_comments:
        max_comments = len(target_comment_records)

    if len(target_worklog_records) > max_worklog:
        max_worklog = len(target_worklog_records)

print('')
print('***********************')
print(max_watchers, '  <-- max watcher headers required')
print(max_comments, '  <-- max comments headers')
print(max_worklog, ' <-- max worklog headers required')



# get the column titles in a list so they can be parsed back into the csv
column_titles_list = list(target_record_set)


watchers_fields = []
watcher_count = 0
for c in range(max_watchers):
    watchers_fields.append('Watcher')
    watcher_countt = watcher_count + 1

comment_fields = []
comment_count = 0
for c in range(max_comments):
    comment_fields.append('Comment')
    comment_count = comment_count + 1


worklog_fields = []
worklog_count = 0
for c in range(max_worklog):
    worklog_fields.append('Worklog')
    worklog_count = worklog_count + 1
    

print('')
print('***********************')
print(len(watchers_fields), ' <-- number of watcher headers created')
print(len(comment_fields), ' <-- number of comment headers created')
print(len(worklog_fields), ' <-- number of worklog headers created')



# build the csv data
csv_row_data = []
for ids in target_issues_list:

    # get a list of the record's data from the processed file
    target_records_processed = list(target_record_set[target_record_set['Issue_ID'] == ids].iloc[0])

    # filter the df to return the target records
    target_watcher_records = watchers_target_set[watchers_target_set.issue_id == ids]
    target_comment_records = comments_target_set[comments_target_set.Issue_ID == ids]
    target_worklog_records = worklog_target_set[worklog_target_set.issueid == ids]


   # build the watcher record and add to the record list
    watcher_record = []
    for row in target_watcher_records.itertuples():
        user_name = row.lower_user_name

        watcher_formatted = f'{user_name}'

        watcher_record.append(watcher_formatted)

    watcher_record_len_processed = make_list_equal_length(watcher_record, len(watchers_fields))


   # build the comment record and add to the record list
    comment_record = []
    for row in target_comment_records.itertuples():
        comment = row.comment_text
        comment_updated = row.comment_updated
        commenter_name = row.commenter_name

        comment_formatted = f'{comment_updated};{commenter_name};{comment}'

        comment_record.append(comment_formatted)

    comment_record_len_processed = make_list_equal_length(comment_record, len(comment_fields))



    # build the worklog record and add to the record list
    worklog_record = []
    for row in target_worklog_records.itertuples():
        worklogbody = row.worklogbody
        STARTDATE = row.STARTDATE
        user_account = row.user_account
        timeworked = row.timeworked

        worklog_formatted = f'{worklogbody};{str(STARTDATE)};{user_account};{str(timeworked)}'

        worklog_record.append(worklog_formatted)

    worklog_record_len_processed = make_list_equal_length(worklog_record, len(worklog_fields))



    output_row = target_records_processed + watcher_record_len_processed + comment_record_len_processed + worklog_record_len_processed


    csv_row_data.append(output_row)


output_headers = column_titles_list + watchers_fields + comment_fields + worklog_fields

output_path = jconfig['output_path'] # <-------------------------------------------- location of the csv that is loaded into the cloud

with open(output_path, 'w', newline='', encoding='utf-8') as f:

    write = csv.writer(f)

    write.writerow(output_headers)
    write.writerows(csv_row_data)

print('CSV build complete')

# Validate the CSV records exist in the cloud

In [ ]:
# validate the CSV records exist in the cloud
output_path = jconfig['output_path'] # <-------------------------------------------- location of the csv that is loaded into the cloud
df = pd.read_csv(output_path, header=None, encoding = "ISO-8859-1")
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0)).reset_index(drop=True)
csv_issue_keys = set(list(df['Issue_key']))
print(len(csv_issue_keys), ' <-- len of csv issue records')

get_all_cloud_issues = jira_session.get_all_issues()
cloud_issue_keys = set(list(get_all_cloud_issues.key))
print(len(get_all_cloud_issues), ' <-- len of the cloud issue records') #key
missing_cloud_records = csv_issue_keys - cloud_issue_keys
print(len(missing_cloud_records), ' <-- number of missing csv records from the cloud')



## Update Comments to the OnPrem State
### Use the API and the local DB to update the comments' internal/external state

In [ ]:
#### NEED TO UPDATE FROM HERE DOWN

df_issue_key_list = list(target_record_set['Issue_key'])
print(len(df_issue_key_list), ' <- len of the csv records that were uploaded to the cloud')
view_filtered_on_keys = commentsdf[commentsdf.Ticket_Key.isin(df_issue_key_list)]
print(len(view_filtered_on_keys), ' <- len of comments records within the records loaded to the cloud')
view_filtered_on_keys_list = list(set(list(view_filtered_on_keys['Ticket_Key'])))

results_output = []        
for k in view_filtered_on_keys_list:
    issue_key = k
    try:
        comment_response = jira_session.get_comments(issue_key)
        if len(comment_response) > 0:
            results_output.append(comment_response)

        else:
            print('************')
            print(f'Review this error for processing {k} because it should have comments and is not processing as expected.')
            print('')

    except:
        print(f'processing error for {k}, you need to see why this is happening.')


concat_results_output = pd.concat(results_output)
print(len(concat_results_output), ' <- sanity check to make sure the same amount of comments loaded to the cloud match the comments pulled from the cloud')




def comment_type(text):
    try:
        if text == '{"internal":true}':
            output ='Internal'

        elif text == '{"internal":"true"}':
            output ='Internal'

        elif text == '{"internal":false}':
            output ='External'
        
        elif text == '{"internal":"false"}':
            output ='External'
        
        return output
    
    except:
        return None



# remove the records that have None as the value in entity_property_json
view_copy = commentsdf.copy()
view_copy = view_copy[~view_copy.entity_property_json.isna()]

# format the field for downstream processing 
view_copy['Comment_Type'] = view_copy.apply(lambda x: comment_type(x.entity_property_json), axis=1)

# slice out the important stuff to merge 
view_slice = view_copy[['comment_text','Ticket_Key', 'Comment_Type', 'unique_comment_id']]



# def parse_comment_json(value):
#     #print(value['content'][0]['content'][0]['text'])
#     try:
#         output = value['content'][0]['content'][0]['text']
#     except:
#         output = 'error parsing content'

#     return output


def parse_comment_json(value):
    #print(value['content'][0]['content'][0]['text'])
    try:
        if len(value['content']) == 1:
            output = value['content'][0]['content'][0]['text']

        elif len(value['content']) > 1:
            for x in value['content']:
                if x['type'] == 'paragraph':
                    y = x['content']
                    for z in y:
                        if z['type'] == 'text':
                            output = z['text']

    except:
        output = 'error parsing content'

    return output


concat_results_output['text'] = concat_results_output.apply(lambda x: parse_comment_json(x.body),axis=1)


print(len(concat_results_output), ' <- sanity check to make sure no errors with parsing the json comments to get the text from the cloud')



'''
Fussy logic assignment of type for records that do not have a direct text match
Step 1:
Use the dataset that contains the records that were not a direct 
text match, then group those records by the unique "Issue_Key" so there is a collective set of
unmatched records from that issue. Then take the set of records that were matched and use that same 
"Issue_Key" to determine the set that was matched for that issue. Then take the comment record set that was
taken from the server that contains the original comments and filter on that same "Isssue_Key". 
Determine from the original set and the matched set what records from the original set have not been matched. 
If all of the remaining, unmatched, original records should be set to "Internal" only then change all of the 
remaining comments that have not been processed to internal

If all remaining, unmatched records, are set to "External" then do nothing

If there is a combination of internal and external records that have not been matched 
then, check if the API text is in the original record text so a fuzzy match can be attempted
by first checking the len() of the len(result_df_error.iloc[0]['body']['content'])
then if greater than 1, for each item check if 'type': 'paragraph' 
then if TRUE then use 'content' to return the list and iterate through the list 
accessing the 'text' to get that text to see if it is in the original text since the 
original text could include the text + attachment details

Default for internal/external combos will be internal as to prevent any 
improper disclosure issues. Users can update as needed. 

'''


# Remove duplicates from view_slice based on the join keys
#view_slice = view_slice.drop_duplicates(subset=['Ticket_Key', 'comment_text'])

result_df = concat_results_output.merge(view_slice, left_on=['Issue_Key', 'text'], right_on=['Ticket_Key', 'comment_text'], how='left')
result_df_matched = result_df[~result_df.unique_comment_id.isna()]
result_df_matched['unique_comment_id']= result_df_matched['unique_comment_id'].astype(int)
#result_df_error = result_df[result_df.Comment_Type.isna()]
result_df_error = result_df[result_df.comment_text.isna()]

print(len(result_df), ' <- len of attempted match by merging the cloud api data with the database data based on text and issue# match')
print(len(result_df), ' <- len of exact matched only records')
print(len(result_df_error), ' <- len of errors only records')



non_matched_list = list(set(list(result_df_error.Issue_Key)))

error_hold = []

for r in non_matched_list:

    error_records = result_df_error[result_df_error.Issue_Key == r]


    original_records = view_slice[view_slice.Ticket_Key == r]
    original_records_list = list(set(list(original_records.unique_comment_id)))

    matched_records = result_df_matched[result_df.Issue_Key == r]
    matched_records_list = list(set(list(matched_records.unique_comment_id)))

    missing_match_list = list(set(original_records) - set(matched_records_list))

    left_over_records = original_records[original_records.unique_comment_id.isin(missing_match_list)]
    left_over_records_list = list(set(list(left_over_records.unique_comment_id)))

    if 'Internal' and not 'External' in left_over_records_list:
        error_records['Comment_Type'] = 'Internal'

    elif 'External' and not 'Internal' in left_over_records_list:
        error_records['Comment_Type'] = 'External'

    # elif 'External' and 'Internal' in left_over_records_list:
    #     # create error so these can be checked
    #     error_records['Comment_Type'] = 'Internal'

    # 'External'

    error_hold.append(error_records)

error_concat = pd.concat(error_hold).reset_index(drop=True)
removed_errors = error_concat[~error_concat.Comment_Type.isna()]
print(removed_errors.Comment_Type.unique(), ' <- sanity check to see if None ARE NOT within the dataset as it should not be as ~.isna() was used')

fixed_error = pd.concat([result_df_matched, removed_errors])
print(len(fixed_error), ' <- len of errors that have been fixed')




check_remaining_errors = error_concat[error_concat.Comment_Type.isna()]
print(len(check_remaining_errors), ' <- len of remaining errors')
print(check_remaining_errors.Comment_Type.unique(), ' <- sanity check to see if only None ARE within the dataset because .isna() was used')




internal_set = fixed_error[fixed_error.Comment_Type == 'Internal']
print(internal_set.Comment_Type.unique(), ' <- sanity check to see if only "Internal" records are being used')

#internal_set = internal_set.iloc[0:5]

def update_comment_status(issue_key, comment_id, comment_type, body):
    if comment_type == None:
        output = 'updated'
    elif comment_type == 'Internal':
        status_type = 'True'
        output = jira_session.update_comment_jsd_public(issue_key, comment_id, status_type, body)
    elif comment_type == 'External':
        output = 'updated'
    return output

print(len(internal_set))

internal_set['update_status'] = internal_set.apply(lambda x: update_comment_status(x.Issue_Key, x.id, x.Comment_Type, x.body), axis=1)




if 'error' in internal_set.update_status.unique():
    print('')
    print('')
    print('******************************************')
    print('')
    print('ACTION REQUIRED --->>>> errors found updating comment status')
    print('')
    print('******************************************')
    print('')
    print('')
elif 'error' not in internal_set.update_status.unique():
    print('')
    print('')
    print('******************************************')
    print('')
    print('NO errors found updating comment status')
    print('')
    print('******************************************')
    print('')
    print('')

## Load Attachements Section
### Using the API to load attachments

In [ ]:

#### this part of the process extracts the required data from the user supplied records
#### regarding what attachments they would like to migrate and provides an error report
#### if there are not issues that allow the attachment to be posted

# loop through the folder that contains the migration records sets from users and extract the contents
attachments_folder_path = jconfig['attachments_folder_path'] # <-------------------------------------------- location to directory that contains excel files from agents that lists the attachments they would like migrated   
attachments_error_path = jconfig['attachments_error_path'] # <-------------------------------------------- location to directory where error records will be stored  
attachments_file_list = list_files_in_folder(attachments_folder_path)
attachments_errors_list = list_files_in_folder(attachments_error_path)



# delete error records if they exist
if len(attachments_errors_list) > 0:
    print('deleting prior error files')
    for i in attachments_errors_list:
        delete_file(i['file_path'])
else:
    print('no prior error files found to delete')


# load the csv with all the jira tickets and use the Issue_key as a filter for the Jira records
output_path = jconfig['output_path'] # <-------------------------------------------- location of the csv that is loaded into the cloud
df = pd.read_csv(output_path, header=None, encoding = "ISO-8859-1")
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0)).reset_index(drop=True)
csv_issue_key = list(df['Issue_key'])


# iterate over the provided attachment excel files and produce an error report
# if there are no matching jira issues
attachments_hold_df = []
for i in attachments_file_list:
    df = pd.read_excel(i['file_path'])
    df = df.drop_duplicates().reset_index(drop=True)
    #df['Issue id'] = df['Issue id'].astype(int)

    # check if the attachments have an issue already created and create an error report if they 
    # do not have an issue to post to
    #
    matching_records = df[df['Issue key'].isin(csv_issue_key)]
    errors = df[~df['Issue key'].isin(csv_issue_key)]
    if len(errors) > 0:
        file_name = i['file_name']
        print('')
        print('**********')
        print(f'{len(errors)} found in {file_name}')
        print('')
        attachment_error_path = attachments_error_path + 'no_matching_issue_'+i['file_name']
        errors.to_excel(attachment_error_path, index=False)

    attachments_hold_df.append(matching_records)
    
attachments_concat = pd.concat(attachments_hold_df)

print('')
print('****************')
print(len(attachments_concat), ' <-- Number of attachments from team members')
print('')
print('')
attachments_concat = attachments_concat.drop_duplicates().reset_index(drop=True)

# parse the excel file to extract the unique ID for each attachment
attachments_concat['attachment_id'] = attachments_concat.apply(lambda x: get_attachment_id(x.Attachment), axis=1)

#### end of processing excel attachment records provided by users



#### start using the excel attachments records to parse the database records

# get a list of the attachment ids so they can be a filter for the database records
attachment_concat_attachment_ids = list(set(list(attachments_concat['attachment_id'])))

# get all attachment records in the database
database_attachment_records = _sys_info_services.get_table_or_view_data('vw_all_jira_attachments_details')
database_attachment_records['ID'] = database_attachment_records['ID'].astype(int)
database_attachment_records['ID'] = database_attachment_records['ID'].astype(str)
# filter only the required records
excel_match_records = database_attachment_records[database_attachment_records.ID.isin(attachment_concat_attachment_ids)]

project_key_list = ['BRS'] # use the Project_Key for bulk load of project records 
all_project_attachments = database_attachment_records[database_attachment_records.Project_Key.isin(project_key_list)]

# get all attachment records after 1/1/2024
database_attachment_records['CREATED'] = pd.to_datetime(database_attachment_records.CREATED)
target_date = pd.to_datetime('1/1/2024')
current_attachments = database_attachment_records[database_attachment_records.CREATED >= target_date] 

# get the entire set 
all_attachment_scope_records = [excel_match_records, all_project_attachments, current_attachments]
target_attachment_records = pd.concat(all_attachment_scope_records)
print(len(target_attachment_records), ' <-- Number of all project attachment records not de-duped')

# drop dups
target_attachment_records_no_dups = target_attachment_records.drop_duplicates(subset=['ID']).reset_index(drop=True)
print(len(target_attachment_records_no_dups), ' <-- Number of no-dups all project attachment records')

# only include attachments that have Jira tickets in the cloud
target_attachment_records_no_dups = target_attachment_records_no_dups[target_attachment_records_no_dups.Ticket_Key.isin(csv_issue_key)].reset_index(drop=True)
print(len(target_attachment_records_no_dups), ' <-- Number of no-dups records that are available in the cloud')




# requirement to remove ALL of required team members' .sql attachments from the migration
name_list = ['T1','T2']
user_list_attachments1 = database_attachment_records[(database_attachment_records.Assignee_User_Display_Name.isin(name_list))]
user_list_attachments2 = database_attachment_records[(database_attachment_records.Reporter_User_Display_Name.isin(name_list))]
user_list_attachments = pd.concat([user_list_attachments1,user_list_attachments2])
target_memetype_df = user_list_attachments[user_list_attachments.MIMETYPE == 'application/x-sql']
target_memetype_list = list(target_memetype_df.ID) 
val_mememtype_df = user_list_attachments.copy()
val_mememtype_df['sql_check'] = val_mememtype_df.apply(lambda x: find_sql_records(x.FILENAME), axis=1)
val_memetype_check = val_mememtype_df[val_mememtype_df.sql_check == 1]
val_memetype_check_list = list(val_memetype_check.ID)

unique_values_in_each_list = unique_elements(target_memetype_list, val_memetype_check_list)

print(len(user_list_attachments), 'len of all users attachements for team members with attachments that need removed')
print(len(target_memetype_df), ' len for all their records when filtered for application/x-sql type files using the MIMETYPE field')
print(len(val_memetype_check), ' len for all their records when validating using a funcation that checks for .sql files')
print(len(unique_values_in_each_list), ' len validating to find if there are unique record ID in each list (sql/meme) indicating issues with ensuring a match for sql records')



# remove all of Nevena and Salam's .sql attachment records from the set 
if len(unique_values_in_each_list) == 0:
    #

    print('')
    print('****************')
    print(len(excel_match_records), ' <-- Number of excel attachment records from team members')
    print(len(all_project_attachments), ' <-- Number of all project attachment records from project approving ALL attachments')
    print((len(excel_match_records)+len(all_project_attachments)+len(current_attachments)), ' <-- Target number of excel and all project attachment records and records after 1/1/2024')
    print(len(target_attachment_records), ' <-- Number of all project attachment records from team members')
    print(len(target_attachment_records_no_dups), ' <-- Number of no-dups all project attachment records from team members BEFORE to removing team member sql records')


    target_attachment_records_no_dups = target_attachment_records_no_dups[~target_attachment_records_no_dups.ID.isin(target_memetype_list)]

    print(len(target_attachment_records_no_dups), ' <-- Number of no-dups all project attachment records from team members AFTER to removing team member sql records')

    print('')
    print('')




    target_attachment_records_no_dups['load_status'] = target_attachment_records_no_dups.apply(lambda x: process_attachments(x.Ticket_Key, x.ID, x.FILENAME, jira_session), axis=1)

    error_loading = target_attachment_records_no_dups[target_attachment_records_no_dups['load_status'] == 0]

    print(len(error_loading), ' <-- then number of error records from the attachment load process.')

else:
    print('errors found with validating team member sql records returning unique values for compairing the two methods for identifing sql records')

# Agent Validation Via Google Drive
### Loading validation records into Google Drive so Agents can review

In [ ]:
# get a listing of the attachements that are in-scope for migrate and use this list to find the out of scope records
attachment_in_scope_list = list(target_attachment_records_no_dups.ID)
attachments_not_in_scope = database_attachment_records[~database_attachment_records.ID.isin(attachment_in_scope_list)]

output_path = jconfig['output_path'] # <-------------------------------------------- location of the csv that is loaded into the cloud
df = pd.read_csv(output_path, header=None, encoding = "ISO-8859-1")
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0)).reset_index(drop=True)
df['Parent_Issue_ID'] = pd.to_numeric(df['Parent_Issue_ID'], errors='coerce').astype('Int64')
df = df.sort_values(by='Parent_Issue_ID', na_position='last')
csv_issue_keys = list(df['Issue_key'])

all_issues = _sys_info_services.get_table_or_view_data('vw_all_jira_issues') # all jira issues
all_issues = all_issues[['Issue_ID', 
                         'Parent_Issue_ID',
                         'Project_Key',
                         'Issue_Status',
                         'Issue_Type',
                         'Issue_Type_Is_SubTask',
                         'Created_Date',
                         'Reporter_UserName',
                         'Creator_UserName',
                         'Assignee_UserName',
                         'Issue_key']]
rds_issues = all_issues[all_issues.Project_Key.isin(['RDS'])]
rds_issues_in_scope = rds_issues[rds_issues.Issue_key.isin(csv_issue_keys)]
rds_issues_not_in_scope = rds_issues[~rds_issues.Issue_key.isin(csv_issue_keys)]


SCOPES = ['https://mail.google.com/',
          'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/spreadsheets']


def update_google_sheets_reporting_views(rds_to_migrated,rds_no_migrate, attachments_to_migrate, attachments_no_migrate):
    '''
    scope:
    updates google sheets that are used for reporting views 
    
    args:
    none
    
    output:
    updates all required gooogle sheets
    
    '''

    key_path = 'jira_migration_cloud_key.json'
    # creditals for gc  
    gc = gspread.service_account(filename=key_path)

    # google sheet reporting views (make sure to add new reports to listing below for processing)
    rds_not_migrated = gc.open_by_url('https://docs.google.com/spreadsheets/d/1smYnX3kArKf8zh2Q3ODlhJK145FV-FhWYXBfWW_BZuE/edit#gid=0')
    rds_not_migrated_worksheet = rds_not_migrated.sheet1

    rds_migrated = gc.open_by_url('https://docs.google.com/spreadsheets/d/1iRCkrEzaJd4L-aX3W5jgkvfL7xSzbWVNAoo2Ks1yUGY/edit#gid=0')
    rds_migrated_worksheet = rds_migrated.sheet1
    
    utt_not_migrated = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Q7SGlTkBD0eMs9HceFzSL5agPLsVQ5eMKUWZwCzpS7A/edit#gid=0')
    utt_not_migrated_worksheet = utt_not_migrated.sheet1

    utt_migrated = gc.open_by_url('https://docs.google.com/spreadsheets/d/1G48rPWCmUGuB7c2b_Rve1ctBz1dfw_XITgr2LFeg35Q/edit#gid=0')
    utt_migrated_worksheet = utt_migrated.sheet1

    attachments_not_migrated = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MGUuUJEZCNYrxLJQYMCFaL8B_ZESS9g2NthbHIoNaAA/edit#gid=0')
    attachments_not_migrated_worksheet = attachments_not_migrated.sheet1

    attachments_migrated = gc.open_by_url('https://docs.google.com/spreadsheets/d/1du1_6iOP3auEfyWzSOi8nQl-Q9wFNm-RGzRwePGo2gs/edit#gid=0')
    attachments_migrated_worksheet = attachments_migrated.sheet1
    

    
    worksheets_list = [rds_not_migrated_worksheet,
                       rds_migrated_worksheet,
                       utt_not_migrated_worksheet,
                       utt_migrated_worksheet,
                       attachments_not_migrated_worksheet,
                       attachments_migrated_worksheet
                       ]
    
    # clear the sheets for loading new data
    for sheets in worksheets_list:
        sheets.clear()
        
     
    # helper funtion for preprocessing the data into a format for google sheets 
    def cast_for_gsheets(df):
        # casting as string if not serializable
        for column, dt in zip(df.columns, df.dtypes):
            if dt.type not in [
                np.int64,
                np.float_,
                np.bool_,
            ]:
                df.loc[:, column] = df[column].astype(str)
        return(df) 

      
    rds_to_migrated_df = cast_for_gsheets(rds_to_migrated)
    rds_no_migrate_df = cast_for_gsheets(rds_no_migrate)
    attachments_to_migrate_df = cast_for_gsheets(attachments_to_migrate)
    attachments_no_migrate_df = cast_for_gsheets(attachments_no_migrate)
    
    
    # add the data to google sheets
    set_with_dataframe(rds_migrated_worksheet, rds_to_migrated_df)
    set_with_dataframe(rds_not_migrated_worksheet, rds_no_migrate_df)
    set_with_dataframe(attachments_migrated_worksheet, attachments_to_migrate_df)
    set_with_dataframe(attachments_not_migrated_worksheet,attachments_no_migrate_df)
    
    print('Google Sheets Reporting Views Have Been Updated')



update_google_sheets_reporting_views(rds_issues_in_scope,rds_issues_not_in_scope, target_attachment_records_no_dups, attachments_not_in_scope)